In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
model_sent = pickle.load(open("embeddings/bert_sent_model.p", "rb"))

2021-11-30 14:07:10.726241: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-30 14:07:10.726364: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
def get_cos_matrix(sent_list1, sent_list2):
    sent_embeddings1 = model_sent.encode(sent_list1)
    sent_embeddings2 = model_sent.encode(sent_list2)
    cos_matrix = cosine_similarity(sent_embeddings1, sent_embeddings2)
    return cos_matrix

In [4]:
def get_similar_sentences(cos_matrix, treshold, sent_list1, sent_list2):
    indexes = np.where((cos_matrix > treshold) & (cos_matrix < 0.999))
    coord_list = list(zip(indexes[0], indexes[1]))
    filename = f"results/similar_sentences_{treshold}.txt"
    with open(filename, 'w') as f:
        f.write(f"{len(coord_list)} entries for cosine similarity greater than {treshold}\n\n")
        for cord in coord_list:
            embd_value = cos_matrix[cord]
            sent1 = sent_list1[cord[0]]
            sent2 = sent_list2[cord[1]]
            if sent1 != sent2:
                f.write(f"{embd_value} \n")
                f.write(f"{sent1} \n")
                f.write(f"{sent2} \n\n")

### Test sentence similarity models on titles

In [5]:
# sentences = [
#     "Exploiting Music Play Sequence for Music Recommendation",
#     "Incremental clustering: The case for extra  clusters ",
#     "To cluster, or not to cluster",
#     "The person box was packed with jelly many dozens of months later.",
#     "He found a leprechaun in his walnut shell.",
#     "Music generation"
# ]

# sentences = [
#     "Should machines evaluate us: opportunities and challenges",
#     "Brain Controlled Musical Improvisation",
#     "EMILY: An Emily Dickinson Machine",
#     "He found a leprechaun in his walnut shell."
# ]

# sentences = [
#     "Measures of Clustering Quality: A Working Set of Axioms for Clustering",
#     "Teaching Computational Creativity",
#     "EMILY: An Emily Dickinson Machine",
#     "He found a leprechaun in his walnut shell."
# ]

sentences1 = [
    "Supporting Computational Music Remixing with a Co-Creative Learning Companion",
    "LyricJam: A system for generating lyrics for live instrumental music",
    "Inspiring Social Creativity in Children with a Digital Tool",
    "He found a leprechaun in his walnut shell."
]

sentences2 = [
    "Humble Creative Machines.",
    "Should Machines Evaluate Us? Opportunities and Challenges",
    "Weighted Clustering: Towards Solving the User’s Dilemma. ",
    "Investors Embrace Gender Diversity, Not Female CEOs: The Role of Gender in Startup Fundraising.",
    "Field Work in Computational Creativity."
]

In [6]:
cos_matrix_test = get_cos_matrix(sentences1, sentences2)

In [7]:
get_similar_sentences(cos_matrix_test, 0.4, sentences1, sentences2)

### Building similar titles in the paper titles dataset

In [8]:
# load paper titles embeddings
titles_embeddings = pickle.load(open("embeddings/bert_titles_embeddings.p", "rb"))

In [9]:
cos_matrix = cosine_similarity(titles_embeddings, titles_embeddings)
cos_matrix.shape

(16859, 16859)

In [10]:
papers = pd.read_csv('papers.csv')
titles = papers['paper_title']

In [11]:
get_similar_sentences(cos_matrix, 0.9, titles, titles)